Connect to your GDrive

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

os.chdir('/content/drive/Shareddrives/WeatherCamera')
!ls -l

Install Packets and Check if we are using a GPU

In [ ]:
!pip install tensorflow==2.8 #
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2 #

import tensorflow as tf
print(tf.__version__)


device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Check if we are using high-RAM

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Extraction Project and Dataset from relative .zips

In [ ]:
# Extraction project
PATH_TO_UNZIP_FILES = "/home"
!unzip -o "/content/drive/Shareddrives/WeatherCamera/Archives/weather-camera-thesis.zip" -d $PATH_TO_UNZIP_FILES
!mv "/home/weather-camera-thesis-main" "/home/weather-camera-thesis"

In [ ]:
# Extraction Dataset - Time to execute: about 13 minutes
PATH_TO_UNZIP_FILES = "/home"
!unzip -o "/content/drive/Shareddrives/WeatherCamera/Archives/Dataset.zip" -d $PATH_TO_UNZIP_FILES

Dataset Annotations

In [ ]:
#If you already have the dataset annotations in Shared Drives
!mkdir /home/weather-camera-thesis/data/
!cp "/content/drive/Shareddrives/WeatherCamera/dataset_annotation.csv" "/home/weather-camera-thesis/data/"

In [ ]:
#If you don't have the dataset annotations in Shared Drives
!python /home/weather-camera-thesis/src/generate_annotations.py

Total Images from Dataset:  2.020.449

* **dry**: 1.167.685 - **wet**: 845.771 - **snow**: 5.386 - **fog**: 1.607 

Percentuali:
* **DRY**:  0.5779
* **WET**:  0.4186
* **SNOW**:  0.0026
* **FOG**:  0.0007
---

Check Dataset Annotations from .csv

In [ ]:
## READ CSV ANNOTATIONS

import os
import csv
import pathlib

HOME_PATH = os.path.abspath('../weather-camera-thesis/')
CSV_PATH = os.path.join(os.path.abspath(os.path.join(pathlib.Path.cwd(), 'data')), "dataset_annotation.csv")
CSV_PATH_COLAB = os.path.abspath('/home/weather-camera-thesis/data/dataset_annotation.csv')

#HOME_PATH = os.path.abspath('../weather-camera-thesis/')
#pathlib.Path.cwd() current folder
#pathlib.Path.cwd().parent current parent folder

classes = ["dry", "wet", "snow", "fog"]
counter = {c:0 for c in classes}
len_data = 0


with open(CSV_PATH_COLAB, 'r') as file:
    reader = csv.reader(file, delimiter = ',')
    for row in reader:
        c = row[1]
        counter[c] += 1
        len_data += 1

DRY_P = counter["dry"]/len_data
WET_P = counter["wet"]/len_data
SNOW_P = counter["snow"]/len_data
FOG_P= counter["fog"]/len_data

#limiti = dry, wet, snow, fog --> 0.30, 0.29, 0.12, 0.29

print("Total Images: ", len_data)
print(counter)
print("Check Sum: ", counter["dry"] + counter["wet"] + counter["snow"] + counter["fog"], "\n")
print("Percentage calculation:")
print("DRY: ", round(DRY_P, 4)*100, "%")
print("WET: ", round(WET_P, 4)*100, "%")
print("SNOW: ", round(SNOW_P, 4)*100, "%")
print("FOG: ", round(FOG_P, 4)*100, "%")
print("Check Percentage: ", (DRY_P + WET_P + SNOW_P + FOG_P)*100, "%")


Partition Dataset from .csv Annotations and Moving Symbolic links-image

In [ ]:
##PARTITION DATASET (ANNOTATIONS)
#Time to execute to create new partition_info: about 2 hours and 30 minutes

from copy import deepcopy
import shutil
import math
import random
from tqdm import tqdm


NEW_DATASET_PATH = os.path.abspath('/home/weather-camera-thesis/data/dataset')
CSV_PATH_COLAB = os.path.abspath('/home/weather-camera-thesis/data/dataset_annotation.csv')
CODE_PATH_COLAB = os.path.abspath('/content/drive/Shareddrives/WeatherCamera/Project Saves/Partition Info')

def print_filenames(path:str, filenames:list)->None:
    with open(path,"w") as f:
        for item in filenames:
            f.write("%s\n" % item)


def check_ds_distribution(counter:dict)->bool:
    # check if the new created dataset respects the orginal dataset distribution #
    #limits = dry, wet, snow, fog --> 0.30, 0.29, 0.12, 0.29

    limits_imported = [DRY_P, WET_P, SNOW_P, FOG_P]
    #limits = [0.30, 0.29, 0.12, 0.29]

    tot = sum(list(counter.values()))
    for i, k in enumerate(list(counter.keys())):
        res = round(counter[k]/tot, 2)
        if not (limits_imported[i]-0.02<=res<=limits_imported[i]+0.02):
            return False
    return True


def split(ratio, data, folder_info):
    print("Annotations of .csv: ", len(data))
    num_images = len(data)
    num_test_images = math.ceil(ratio*num_images)
    print("Train: {}, Test: {}".format(num_images-num_test_images, num_test_images))

    classes = ["dry", "wet", "snow", "fog"]
    counter = {c:0 for c in classes}
    done = False

    while not done:
        print("TEST")
        images_copy = deepcopy(data)
        for k in list(counter.keys()): counter[k]=0 # clear dict
        filenames = []
        for i in tqdm(range(num_test_images)):
            idx = random.randint(0, len(images_copy)-1)
            filename = images_copy[idx]
            filenames.append(filename)
            c = filename[1]
            counter[c] +=1
            images_copy.remove(filename)
        done = check_ds_distribution(counter)
        #print("TEST counter: ", counter)

        # if distribution is respected, process remaining images-annotations for training set
        if done:
            print("TRAIN")
            for k in list(counter.keys()): counter[k]=0 # clear dict
            for filename in tqdm(images_copy):
                c = filename[1]
                counter[c] +=1
            # print(counter)
            done = done and check_ds_distribution(counter)
            #print("TRAIN counter: ", counter)
    
    print("Split done!")
    print_filenames(os.path.join(folder_info, "test_filenames.txt"), filenames)
    print_filenames(os.path.join(folder_info, "train_filenames.txt"), images_copy)
    return filenames, images_copy


def adjust_read(folder, txt_file):
    with open(os.path.join(folder,txt_file),"r") as f:
        val_filenames = f.readlines()

    val_filenames = [f.rstrip() for f in val_filenames]
    new_filenames = []

    for f in val_filenames:
        path1, path2 = f.split(',')
        filename = [path1[2:-1],path2[2:-2]]
        new_filenames.append(filename)
    return new_filenames


def recover_filenames(folder:str)->list:
    # recover annotations for training and validation from file"
    val_filenames = adjust_read(folder, "test_filenames.txt")
    train_filenames = adjust_read(folder, "train_filenames.txt")

    return val_filenames, train_filenames


def move_images(val_filenames:list, train_filenames:list, dest:str):
    # starting from lists of filename, move annotations in the right folders #
    train_dir = os.path.join(dest, 'train')
    test_dir = os.path.join(dest, 'test')

    #Creation folders
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
        os.makedirs(os.path.join(train_dir, "dry"))
        os.makedirs(os.path.join(train_dir, "fog"))
        os.makedirs(os.path.join(train_dir, "wet"))
        os.makedirs(os.path.join(train_dir, "snow"))


    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
        os.makedirs(os.path.join(test_dir, "dry"))
        os.makedirs(os.path.join(test_dir, "fog"))
        os.makedirs(os.path.join(test_dir, "wet"))
        os.makedirs(os.path.join(test_dir, "snow"))

    print("TEST")
    for f in tqdm(val_filenames):
        #shutil.copy(f[0], os.path.join(test_dir, f[1]))
        img = f[0].rsplit('/', 1)
        os.symlink(f[0], test_dir+"/"+f[1]+"/"+str(img[-1]))
    
    print("TRAIN")
    for f in tqdm(train_filenames):
        #shutil.copy(f[0], os.path.join(train_dir, f[1]))
        img = f[0].rsplit('/', 1)
        os.symlink(f[0], train_dir+"/"+f[1]+"/"+str(img[-1]))


if __name__ == '__main__':

    with open(CSV_PATH_COLAB, newline='') as f:
        reader = csv.reader(f)
        data = list(reader)
        
    folder = os.path.join(CODE_PATH_COLAB, "partition_info1") ######

    if not os.path.exists(folder):
        os.makedirs(folder)
        ratio = 0.1
        print("Create new Partition Dataset with split ratio: {}".format(ratio))
        val_filenames, train_filenames = split(ratio, data, folder)

    else:
        print("Found an existing Partition Info: ", folder)
        val_filenames, train_filenames = recover_filenames(folder)
        print("Split recovered !")

    print("Moving symbolic links-image...")
    move_images(val_filenames, train_filenames, NEW_DATASET_PATH)


Build Model: MobileNetV2

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'    # Suppress TensorFlow logging (1)
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers
from tensorflow import keras


INPUT_SHAPE = (224,224,3)
IMG_SIZE = 224
N_CLASSES = 4 #dry, wet, snow, fog
N_LAYERS_TO_TRAIN = 10 #to modify

img_augmentation = keras.Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(mode="horizontal"),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

def build_model(freeze:bool=None)->tf.keras.Model:

    #inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)) #
    #x = img_augmentation(inputs)                         #
    base_model = MobileNetV2(INPUT_SHAPE, include_top=False, weights='imagenet') #input_tensor=x # 154 layers
    if freeze is not None:
        if freeze: base_model.trainable = False
        else:
            for layer in base_model.layers:
                if not isinstance(layer, layers.BatchNormalization):
                    layer.trainable = True
    x = layers.GlobalAveragePooling2D()(base_model.output)
    #x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x) #DA RIMUOVERE INIZIALMENTE
    model = layers.Dense(N_CLASSES, 'softmax')(x)
    final_model = tf.keras.Model(inputs=base_model.input, outputs=model)
    #print(final_model.summary())
    return final_model

Setting of the Training:

In [ ]:
from sklearn.utils import class_weight
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

NEW_DATASET_PATH = os.path.abspath('/home/weather-camera-thesis/data/dataset/train')

n_experiment = "?_experiment" #specify the experiment to execute

CHECKPOINT_PATH = "/content/drive/Shareddrives/WeatherCamera/Project Saves/{}/checkpoint/".format(n_experiment)
LOGS_PATH = "/content/drive/Shareddrives/WeatherCamera/Project Saves/{}/logs/".format(n_experiment)
TB_LOGS_PATH = "/content/drive/Shareddrives/WeatherCamera/Project Saves/{}/tb_logs/".format(n_experiment)
DOC_PATH = "/content/drive/Shareddrives/WeatherCamera/Project Saves/{}/doc".format(n_experiment)


if not os.path.exists(CHECKPOINT_PATH): os.makedirs(CHECKPOINT_PATH)
if not os.path.exists(LOGS_PATH): os.makedirs(LOGS_PATH)
if not os.path.exists(TB_LOGS_PATH): os.makedirs(TB_LOGS_PATH)
if not os.path.exists(DOC_PATH): os.makedirs(DOC_PATH)    

def train():

    TRAIN_DIM = 1454724 #
    VAL_DIM = 363680 #
    print("TRAIN Dim: ", TRAIN_DIM, ", VAL Dim: ", VAL_DIM)
    CLASSIFICATOR_INPUT_SIZE = (224,224)

    batch_size = 32
    epochs = 15
    patience= 10
    learning_rate = 0.00003

    freeze_model = False #to check
    resume_training = False #to check
    model_path = os.path.abspath("/") #to check

    # model
    model = build_model(freeze_model)
    if resume_training:
        model.load_weights(model_path)
    
    #lr_schedule = tf.keras.optimizers.schedules.CosineDecay(args.learning_rate, int(args.epochs*(TRAIN_DIM/args.batch_size)))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
                loss=tf.keras.losses.categorical_crossentropy, 
                metrics=[tf.keras.metrics.categorical_accuracy]
                )

    train_datagen = ImageDataGenerator(validation_split=0.20)
    #test_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow_from_directory(
        directory=NEW_DATASET_PATH,
        target_size=CLASSIFICATOR_INPUT_SIZE,
        #color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        subset='training',
        shuffle=True,
        seed=556
    )

    valid_generator = train_datagen.flow_from_directory(
        directory=NEW_DATASET_PATH,
        target_size=CLASSIFICATOR_INPUT_SIZE,
        #color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        subset='validation',
        shuffle=True,
        seed=556
    )

    class_weights = dict(zip(np.unique(valid_generator.classes), class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(valid_generator.classes), y=valid_generator.classes))) 
    print("Class Weights: ", class_weights)

    #resize images
    #train_ds = train_ds.map(lambda image, label: (tf.image.resize(image, CLASSIFICATOR_INPUT_SIZE), label))
    #val_ds = val_ds.map(lambda image, label: (tf.image.resize(image, CLASSIFICATOR_INPUT_SIZE), label))
    #train_ds = train_ds.prefetch(buffer_size=32)
    #val_ds = val_ds.prefetch(buffer_size=32)

    # callbacks --> val_loss, prima c'era "val_categorical_accuracy"
    filepath = 'model-epoch_{epoch:02d}.hdf5'
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(os.path.join(CHECKPOINT_PATH, filepath), save_weights_only=True, verbose=1, save_best_only=True)
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=patience,verbose=1,monitor='val_loss',mode='auto')
    history_logger_cb = tf.keras.callbacks.CSVLogger((os.path.join(LOGS_PATH, 'training_log.csv')), separator=",", append=True)
    tensorboad_cb = tf.keras.callbacks.TensorBoard(log_dir=TB_LOGS_PATH, write_graph=False)
    callbacks = [checkpoint_cb,early_stopping_cb,history_logger_cb,tensorboad_cb]

    hist = model.fit(train_generator, validation_data = valid_generator, epochs=epochs, verbose = 1, callbacks=callbacks, shuffle = False, class_weight=class_weights,
                steps_per_epoch=int(np.ceil(TRAIN_DIM / batch_size)),validation_steps=int(np.ceil(VAL_DIM / batch_size)))
    

In [ ]:
## Execute the Training
train()

Plotting history of the training

In [ ]:
import csv
%matplotlib inline
import matplotlib.pyplot as plt

N_CLASSES = 4
image_size = (224, 224)

def plot_hist(csv_path:str, save:bool, imgs_path:str=None)->list:
    with open(csv_path, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        hist ={"epoch":[], "categorical_accuracy":[], "val_categorical_accuracy":[],"loss":[],"val_loss":[]}
        for row in csv_reader:
            hist["epoch"].append(float(row["epoch"]))
            hist["categorical_accuracy"].append(float(row["categorical_accuracy"]))
            hist["val_categorical_accuracy"].append(float(row["val_categorical_accuracy"]))
            hist["loss"].append(float(row["loss"]))
            hist["val_loss"].append(float(row["val_loss"]))

        n_epochs = len(hist["epoch"])

        plt.figure() 
        plt.title("LOSS")
        plt.plot(hist["loss"], color='blue', label='loss')
        plt.plot(hist["val_loss"], color='orange', label='Val_Loss')
        plt.xticks(range(0,n_epochs))
        plt.ylabel("loss")
        plt.xlabel("epoch")
        plt.legend(["train", "validation"], loc="best")

        if save: 
            path_loss_fig = imgs_path+"/loss_{}.png".format(n_epochs)
            plt.savefig(path_loss_fig)
        else: plt.show()

        plt.figure()
        plt.title("ACCURACY")
        plt.plot(hist["categorical_accuracy"], color='blue', label='Accuracy')
        plt.plot(hist["val_categorical_accuracy"], color='orange',label='Val_accuracy')
        plt.xticks(range(0,n_epochs))
        plt.ylabel("accuracy")
        plt.xlabel("epoch")
        plt.legend(["train", "validation"], loc="best")
        
        if save: 
            path_acc_fig = imgs_path+"/acc_{}.png".format(n_epochs)
            plt.savefig(path_acc_fig)
        else: plt.show()

        if save: return [path_loss_fig, path_acc_fig]

In [ ]:
## Plot
n_experiment = "?_experiment" #specify the experiment to visualize

TRAIN_LOG_PATH = "/content/drive/Shareddrives/WeatherCamera/Project Saves/{}/logs/training_log.csv".format(n_experiment)
DOC_PATH = "/content/drive/Shareddrives/WeatherCamera/Project Saves/{}/doc".format(n_experiment)


plot_hist(TRAIN_LOG_PATH, save=True, imgs_path=DOC_PATH)

Testing of the Model

In [ ]:
import pandas as pd
from tqdm import tqdm
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'    # Suppress TensorFlow logging (1)
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score, balanced_accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import keras


CLASSIFICATOR_INPUT_SIZE = (224,224)

n_experiment = "?_experiment" #specify the experiment to test
n_partition_info = "partition_info?" #specify the partition_info to test

model_path = "/" #to check
TEST_SET_PATH = os.path.abspath('/content/drive/Shareddrives/WeatherCamera/Project Saves/Partition Info/{}/test_filenames.txt'.format(n_partition_info))
OUTPUT_PATH = os.path.abspath('/content/drive/Shareddrives/WeatherCamera/Project Saves/{}/doc/results.txt'.format(n_experiment))

category_index_classifier = {"dry": 0, "fog": 1, "snow": 2, "wet": 3}

"""
## Run inference on new data
"""

def inference_data(model_path):
    
    print("Loading model...")
    model = build_model()
    model.load_weights(model_path)
    print("Loading model...DONE")

    y_true = []
    y_pred = []
    
    with open(TEST_SET_PATH) as f:
        lines = f.readlines()
        lines = [f.rstrip() for f in lines]

    print("Inference data:")
    for f in tqdm(lines):
        path1, path2 = f.split(',')
        filename = [path1[2:-1],path2[2:-2]]

        img_path = filename[0]

        img = keras.preprocessing.image.load_img(img_path, target_size=CLASSIFICATOR_INPUT_SIZE)
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  # Create batch axis

        true_label = category_index_classifier[filename[1]]
        y_true.append(true_label)

        predictions = model.predict(img_array)
        score = predictions[0]
        pred_label = np.argmax(score)
        y_pred.append(pred_label)


    labels = list(category_index_classifier.keys())

    print("Building confusion matrix...")
    matrix = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=labels)
    disp.plot()
    plt.savefig(os.path.abspath('/content/drive/Shareddrives/WeatherCamera/Project Saves/{}/doc/cm_test.png'.format(n_experiment)))
    matrix = pd.DataFrame(matrix, index=["true:{}".format(x) for x in labels], columns=["pred:{}".format(x) for x in labels])
    print("Building confusion matrix...DONE")
    print("Confusion Matrix:\n", matrix)
    
    print("Computing Metrics...")
    acc = accuracy_score(y_true, y_pred)
    bal_accuracy = balanced_accuracy_score(y_true, y_pred)
    rec_score = recall_score(y_true, y_pred, average='weighted')
    prec_score = precision_score(y_true, y_pred, average='weighted')
    f1score = f1_score(y_true, y_pred, average='weighted')
    print("Computing Metrics...DONE")
    print("Accuracy Score: ", round(acc, 4))
    print("Balanced Accuracy Score: ", round(bal_accuracy, 4))
    print("Recall Score: ", round(rec_score, 4))
    print("Precision Score: ", round(prec_score, 4))
    print("F1-Score: ", round(f1score, 4))

    with open(OUTPUT_PATH,"w") as f:
        f.write("CONFUSION MATRIX\n"+matrix.to_string())
        f.write("\n\nACCURACY:\n"+str(round(acc, 4)))
        f.write("\n\nBALANCED ACCURACY:\n"+str(round(bal_accuracy, 4)))
        f.write("\n\nPRECISION:\n"+str(round(prec_score, 4)))
        f.write("\n\nRECALL:\n"+str(round(rec_score, 4)))
        f.write("\n\nF1-SCORE:\n"+str(round(f1score, 4)))


inference_data(model_path)